In [1]:
from Utils.loadset import getDataSet
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import tensorflow_probability as tfp
import os
physical_devices = tf.config.list_physical_devices('GPU') 
print("Num GPUs:", len(physical_devices)) 

from trainer import Trainer
try:
    from Utils.connection_cfg import *
except Exception as e:
    PSWD = None
    USRN = None
    
from Utils.Data import provideData
from Utils.transform import cutOut

Num GPUs: 1
Num GPUs Available: 1


Using TensorFlow backend.


In [2]:
#slices = [272,304,272,304]
slices = [256,320,256,320]
cutOutFrame = cutOut(slices)

PRETRAINING_TRANSFORMATIONS = [cutOutFrame]
DatasetFolder = "./Data/RAW"
PathToData = os.path.join(DatasetFolder,"MonthPNGData")
dimension = (64,64)
channels = 7
optimizer = Adam( lr = 1e-3 )
batch_size=25
year = [2015,2016,2017]
train, test = provideData(dimension=dimension,
                          batch_size=batch_size,
                          channels = channels,
                          timeToPred = 30,
                          fillSteps = True,
                          year = year,
                          transform=None,
                          preTransformation=PRETRAINING_TRANSFORMATIONS)



tfd = tfp.distributions
def NLL(y_true, y_hat):
    return -y_hat.log_prob(y_true)



Found Year :  2015 => won't download this year again... please check for consistency
Found Year :  2016 => won't download this year again... please check for consistency
Found Year :  2017 => won't download this year again... please check for consistency
Finished Loading Dataset
 
[DEBUG] Data
[DEBUG] time to predict:  30
SORTTING OUUUT 64784
[DEBUG] Data
[DEBUG] time to predict:  30
SORTTING OUUUT 25133


In [36]:


tfd = tfp.distributions
def NLL(y_true, y_hat):
    return -y_hat.log_prob(y_true)


def ZeroInflated_negativ_Binomial(input_shape,
           n_predictions=1,
           simpleclassification=None,
           flatten_output=False,
           activation_hidden="relu",
           activation_output="relu"):


    

    inputs = Input(shape=input_shape) 

    conv01 = Conv2D(28, kernel_size=(3, 3), padding="same")(inputs)       # 10 x 64x64
    conv01 = Activation(activation_hidden)(conv01)
    conv01_pool = MaxPooling2D((2, 2), strides=(2, 2))(conv01)            # 10 x 32x32


    conv02 = Conv2D(60, kernel_size=(3, 3), padding="same")(conv01_pool)  # 20 x 32x32
    conv02 = Activation(activation_hidden)(conv02)
    conv02_pool = MaxPooling2D((2, 2), strides=(2, 2))(conv02)            # 20 x 16x16


    conv03 = Conv2D(100, kernel_size=(3, 3), padding="same")(conv02_pool)  # 20 x 16x16
    conv03 = Activation(activation_hidden)(conv03)
    conv03_pool = MaxPooling2D((2, 2), strides=(2, 2))(conv03)            # 20 x 8x8


    conv04 = Conv2D(120, kernel_size=(3, 3), padding="same")(conv03_pool)  # 20 x 8x8
    conv04 = Activation(activation_hidden)(conv04)
    conv04_pool = MaxPooling2D((2, 2), strides=(2, 2))(conv04)            # 20 x 4x4


    ### UPSAMPLING:
    up04 = UpSampling2D((2, 2))(conv04_pool)    # 20 x 8x8
    up04 = concatenate([conv04, up04], axis=3)  # 20+20 x 8x8


    up03 = UpSampling2D((2, 2))(up04)           # 40 x 16x16
    up03 = concatenate([conv03, up03], axis=3)  # 20+40 x 16x16


    up02 = UpSampling2D((2, 2))(up03)           # 60 x 32x32
    up02 = concatenate([conv02, up02], axis=3)  # 20+60 x 32x32


    up01 = UpSampling2D((2, 2))(up02)           # 80 x 64x64
    up01 = concatenate([conv01, up01], axis=3)  # 10+80 x 64x64

    
    layer = Conv2D(64, (3,3), activation="relu", padding="same")(up01)  # 1 x 64x64
    layer = Conv2D(18, (7,7), activation="tanh", padding="same")(layer)
    input_dist= layer

    output_dist = tfp.layers.DistributionLambda(name="DistributionLayer",
                                                make_distribution_fn=lambda t: tfp.distributions.Independent(
                                                tfd.Mixture(
                                                    cat=tfd.Categorical(tf.stack([1-tf.math.sigmoid(t[...,:6]), tf.math.sigmoid(t[...,:6])],axis=-1)),
                                                    components=[tfd.Deterministic(loc=tf.zeros_like(t[...,:6])),
                                                    tfp.distributions.NegativeBinomial(
                                                    total_count=tf.math.softplus(t[..., 6:12]), probs=tf.math.sigmoid(t[..., 12:]) ),])
                                                    ,name="ZeroInflated_Binomial",reinterpreted_batch_ndims=0 ))
    

    output = output_dist(input_dist)
    model = Model(inputs=inputs, outputs=output)
    return model



In [37]:
modelPath = "./model_data/ZeroInflated_negativ_Binomial_fillsteps"
if not os.path.exists(modelPath):
    os.mkdir(modelPath)

model = ZeroInflated_negativ_Binomial((*dimension,7))
model.compile(loss=NLL, optimizer=Adam( lr= 1e-4 ))
model.summary()

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 64, 64, 7)]  0                                            
__________________________________________________________________________________________________
conv2d_70 (Conv2D)              (None, 64, 64, 28)   1792        input_13[0][0]                   
__________________________________________________________________________________________________
activation_48 (Activation)      (None, 64, 64, 28)   0           conv2d_70[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_48 (MaxPooling2D) (None, 32, 32, 28)   0           activation_48[0][0]              
___________________________________________________________________________________________

In [38]:
from tensorflow.keras.callbacks import *

modelname = os.path.join(modelPath,'model-{epoch:03d}-{loss:03f}-{val_loss:03f}.h5')
checkpoint = ModelCheckpoint(modelname, verbose=1, monitor='val_loss',save_best_only=True, mode='auto') 
history = model.fit(train,validation_data=test,epochs=20,callbacks=checkpoint)

Epoch 1/20
2589/2590 [============================>.] - ETA: 0s - loss: 1.2441
Epoch 00001: val_loss improved from inf to 1.25128, saving model to ./model_data/ZeroInflated_negativ_Binomial_fillsteps/model-001-1.244108-1.251285.h5
2590/2590 [==============================] - 125s 48ms/step - loss: 1.2441 - val_loss: 1.2513
Epoch 2/20
2589/2590 [============================>.] - ETA: 0s - loss: 1.2096
Epoch 00002: val_loss improved from 1.25128 to 1.24509, saving model to ./model_data/ZeroInflated_negativ_Binomial_fillsteps/model-002-1.209516-1.245094.h5
2590/2590 [==============================] - 127s 49ms/step - loss: 1.2095 - val_loss: 1.2451
Epoch 3/20
2589/2590 [============================>.] - ETA: 0s - loss: 1.2075
Epoch 00003: val_loss improved from 1.24509 to 1.23697, saving model to ./model_data/ZeroInflated_negativ_Binomial_fillsteps/model-003-1.207474-1.236970.h5
2590/2590 [==============================] - 126s 49ms/step - loss: 1.2075 - val_loss: 1.2370
Epoch 4/20
2589/2

In [ ]:

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.pyplot import figure
import numpy as np
%matplotlib inline
#model = t.model
pred = None
label = None
pred = 10
test.on_epoch_end()
mean = []
stdev = []
label = []
predictions = []
for x,y in test:
    for i in range(batch_size):
        if y[i,:,:,:].max() > 0:
            pred = model(np.array([x[i,:,:,:]]))
            mean.append(pred.mean())
            stdev.append(pred.stddev())
            label.append(y[i,:,:,:])
            predictions.append((pred,y[i:,:,:]))
            if len(label) == 10:
                break
        #mean = model(x).mean()
        #std = model(x).stddev()
        #label = y
        break
    if len(label) == 10:
                break

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.pyplot import figure
import numpy as np
%matplotlib inline
def plotHistory():

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()
plotHistory()

In [ ]:
fig, axes = plt.subplots(10, 3, figsize=(16,8*10),dpi=64)
#fig.set_title(["mean","stdev","label"])
for batch,img in enumerate(mean):
    if batch == 10:
        break
    axes[batch,1].imshow(img[0,:,:,-1],cmap="gray")
    axes[batch,0].imshow(stdev[batch][0,:,:,-1],cmap="gray")
    axes[batch,2].imshow(label[batch][:,:,-1],cmap="gray")

In [ ]:
index = 0

p,l = predictions[index]
l = l[0]
ones = np.ones_like(l)
values = 100
value_array = np.repeat(ones,values,axis=-1)
z = value_array.shape[-1]
a = np.arange(values)
for i in range(values):
    value_array[:,:,i] *= i
    
probs_values = np.array(p.prob(value_array)[0])

max_value = l.max()
idcs = np.where(l == max_value)

x_i,y_i = idcs[0],idcs[1]
y_ = probs_values[x_i,y_i,:][0]
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

true_v = np.zeros_like(y_)
true_v[max_value] = 1
sns.set(color_codes=True)
sns.set(rc={'figure.figsize':(20,8.27)})
sns.barplot(a,y_)
sns.barplot(a,true_v)
plt.show()


In [ ]:

plt.imshow(l[:,:,0])
plt.show()

plt.imshow(probs_values[:,:,0])
plt.show()


In [ ]:
#def simpleBaseline(label,prediction):
    
#simpleBaseline
def dist2Classes(dist):
    zeros = np.zeros(dist.shape)
    # Wahrscheinlichkeit für kein Regen
    rain = np.array(dist.cdf(zeros))
    
    # Wahrscheinlichkeit kein Regen größer 50%, 
    mask = (rain > 0.5)
    
    return mask

def labelToMask(label):
    # Gibt Maske mit kein Regen zurück
    return (label == 0)

true_pos = np.zeros((1,64,64,1))
true_neg = true_pos.copy()
false_pos = true_pos.copy()
false_neg = true_pos.copy()
rain_total = 0
total = 0

simple_true_pos = np.zeros((1,64,64,1))
simple_true_neg = true_pos.copy()
simple_false_pos = true_pos.copy()
simple_false_neg = true_pos.copy()
for i,(x,y) in enumerate(test):
    batchsize = y.shape[0]
    
    for batch in range(batchsize):
        print("{}/{}".format((i*batchsize)+batch,len(test)*batchsize),end="\r")
        pred = model(np.array([x[batch,:,:,:]]))
        
        pred_mask = dist2Classes(pred)
        labl_mask = labelToMask(y[batch,:,:,:])
        #              Kein Regen Predicted & Label = Kein Regen
        false_neg  += (pred_mask == True)    & (labl_mask == True)
        #              Regen Predicted       & Label = Kein Regen
        true_neg   += (pred_mask == False)   & (labl_mask == True)
        #              Regen Predicted       & Label = Regen
        true_pos   += (pred_mask == False)   & (labl_mask == False)
        #              Kein Regen Predicted  & Label = Regen
        false_pos  += (pred_mask == True)    & (labl_mask == False)
        rain_total += (~labl_mask).sum()
        total += labl_mask.sum() + (~labl_mask).sum()
        
        rain_atm = np.array(x[batch,:,:,-1])
        label_simple = labelToMask(rain_atm)
        
        # Simple Baseline
        
        rain_atm = x[batch,:,:,-1:]
        pred_simple = labelToMask(rain_atm)
        #              Kein Regen Predicted & Label = Kein Regen
        simple_false_neg  += (pred_simple == True)    & (labl_mask == True)
        #              Regen Predicted       & Label = Kein Regen
        simple_true_neg   += (pred_simple == False)   & (labl_mask == True)
        #              Regen Predicted       & Label = Regen
        simple_true_pos   += (pred_simple == False)   & (labl_mask == False)
        #              Kein Regen Predicted  & Label = Regen
        simple_false_pos  += (pred_simple == True)    & (labl_mask == False)
        

In [ ]:
import seaborn as sn
import pandas as pd
plt.figure(figsize = (4,4))

mat = np.array([[false_neg.sum(),false_pos.sum()],[true_neg.sum(),true_pos.sum()]])
total = mat.sum()
df_cm = pd.DataFrame(mat/total,index = ["Actual 0","Actual 1"],columns=["Predicted 0","Predicted 1"])
sn.heatmap(df_cm, annot=True,vmin=0.0, vmax=1.0)
plt.show()

In [ ]:
plt.figure(figsize = (4,4))
print("Simple Baseline")
mat_simple = np.array([[simple_false_neg.sum(),simple_false_pos.sum()],[simple_true_neg.sum(),simple_true_pos.sum()]])
total = mat.sum()
df_cm = pd.DataFrame(mat_simple/total,index = ["Actual 0","Actual 1"],columns=["Predicted 0","Predicted 1"])
sn.heatmap(df_cm, annot=True,vmin=0.0, vmax=1.0)
plt.show()

In [ ]:

print("Regen total: {}\nKein Regen total: {}\nVerhältnis: {:.4f}\n".format(rain_total,total-rain_total,rain_total/(total)))